### Load VGG19 Model

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import tensorflow as tf
model = tf.keras.applications.vgg19.VGG19(weights='imagenet')

### Load an Image

In [ ]:
img = tf.keras.preprocessing.image.load_img('dog.jpeg')
img

### Prediction

In [ ]:
import numpy as np

In [ ]:
#Prepare image prediction
img_array = tf.keras.preprocessing.image.img_to_array(img.resize((224, 224)))

#Normalize the image
img_array[:,:,0] -= 103.939
img_array[:,:,1] -= 116.779
img_array[:,:,2] -= 123.68

#Create a batch
img_array = np.expand_dims(img_array, axis=0)

#Get the prediction
prediction = model.predict(img_array)

#Get the index with highest probability
pred_index = np.argmax(prediction[0])

#Decode prediction
prediction = tf.keras.applications.vgg19.decode_predictions(prediction)

pred_index, prediction

ImageNet Class IDs
- https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a

### Building Saliency Maps

Access model input and output

In [ ]:
input_img = model.input
model_output = model.output

What should be the loss?

In [ ]:
loss = tf.keras.backend.mean(model_output[:,258])

Calculate Gradients

In [ ]:
#Gradient of loss w.r.t input image
grads = tf.keras.backend.gradients(loss, input_img)[0]

#Gradient normalization
K = tf.keras.backend
grads /= K.sqrt(K.mean(K.square(grads))) + K.epsilon()

#Define function to run Graph
iterate = K.function([input_img], [loss, grads])

In [ ]:
loss_value, grads_value = iterate([img_array])

### Plot Gradients as Saliency Map

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(15,12))
plt.subplot(1,2,1)
plt.imshow(img.resize((224,224)))
plt.subplot(1,2,2)
plt.imshow(grads_value[0], cmap='jet')
plt.show()